In [130]:
import pandas as pd
import numpy as np

In [145]:
nodes_path = "./data/nodes.csv"
edges_path = "./data/edges.csv"

pods = [
    "adservice", "cartservice", "checkoutservice", "currencyservice", "emailservice", "frontend", "paymentservice", "productcatalogservice", "recommendationservice", "redis", "shippingservice",
]

window = 60
horizont = 30
steps = 10

In [70]:
nodes_df = pd.read_csv(nodes_path)
edges_df = pd.read_csv(edges_path)

In [147]:
len(nodes_df)

1440

In [71]:
nodes_df = nodes_df.sort_values("timestamp").sort_values("timestamp")
edges_df = edges_df.sort_values("timestamp").sort_values("timestamp")

In [209]:
X_nodes = []
y_nodes = []

for i in range(0, len(nodes_df) - window - horizont + 1, steps):
    node_window = nodes_df.iloc[i : i + window]
    node_horizont = nodes_df.iloc[i + window : i + window + horizont]
    
    x = []
    y = []
    for pod in pods:
        x.append(node_window[[f"{pod}_cpu", f"{pod}_ram"]].to_numpy())
        y.append(node_horizont[[f"{pod}_cpu", f"{pod}_ram"]].to_numpy())
    X_nodes.append(np.array(node_x).swapaxes(0, 1))
    y_nodes.append(np.array(node_y).swapaxes(0, 1))

In [211]:
np.array(X_nodes).shape, np.array(y_nodes).shape

((136, 60, 11, 2), (136, 30, 11, 2))

In [213]:
np.savez("./data/node_features.npz", X=X_nodes, y=y_nodes)

In [214]:
np.load("./data/node_features.npz")['X'].shape

(136, 60, 11, 2)

## Edges

In [215]:
edges_timed = edges_df.groupby("timestamp")

In [221]:
X_edges = []
y_edges = []
graph = []

for k in range(0, len(nodes_df) - window - horizont + 1, steps):
    edges = nodes_df.iloc[k : k + window + horizont]
    if len(graph) == window + horizont:
        edges = edges.iloc[-steps:]
        graph = graph[steps:]

    for t in edges["timestamp"]:
        edges_info = edges_timed.get_group(t)
        A = np.zeros((len(pods), len(pods)))
        for i in range(len(pods)):
            for j in range(len(pods)):
                from_pod = pods[i]
                to_pod = pods[j]
                
                query = edges_info.query("`from` == @from_pod and `to` == @to_pod")
                if len(query) > 0:
                    A[i, j] = query["value"].item()
        graph.append(A)
    X_edges.append(graph[:window])
    y_edges.append(graph[window:])

In [224]:
np.savez("./data/edges_features.npz", X=X_edges, y=y_edges)